### 对比总结

| 特性                | XPath               | JSONPath            | BeautifulSoup          |
|---------------------|---------------------|---------------------|------------------------|
| **数据格式**         | HTML/XML           | JSON               | HTML/XML              |
| **语法复杂度**       | 高（需学习路径表达式） | 中（类似XPath）     | 低（Pythonic API）     |
| **性能**             | 高（配合`lxml`）    | 中                  | 中/低（依赖解析器）     |
| **容错性**           | 低                 | -（JSON本身结构化） | 高（自动修复HTML）     |
| **典型使用场景**     | 精准提取复杂HTML元素 | 提取多层嵌套JSON数据 | 快速解析不规范HTML     |

## xpath

xpath基本语法：  
1.路径查询  
//：查找所有子孙节点，不考虑层级关系  
/ ：找直接子节点  
2.谓词查询  
//div[@id]  
//div[@id="maincontent"]  
3.属性查询  
//@class  
4.模糊查询  
//div[contains(@id, "he")]  
//div[starts‐with(@id, "he")]  
5.内容查询  
//div/h1/text()  
6.逻辑运算  
//div[@id="head" and @class="s_down"]  
//title | //price  

### 获取百度网站的百度一下

In [ ]:

from lxml import etree
from urllib.request import urlopen, Request

url = "https://www.baidu.com/"

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, likeGecko) Chrome/74.0.3729.169 Safari/537.36'
}

request = Request(url, headers=headers)
response = urlopen(request)
content = response.read().decode('utf-8')

tree = etree.HTML(content)
result = tree.xpath("//input[@id='su']/@value")[0]
print(result)

百度一下


### 案例：站长图片素材下载

In [ ]:
from urllib.request import urlretrieve

def create_request(page):
    if page == 1:
        url = "https://sc.chinaz.com/tupian/hefumeinv_2.html"
    else:
        url = "https://sc.chinaz.com/tupian/hefumeinv_{}.html".format(page)

    headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, likeGecko) Chrome/74.0.3729.169 Safari/537.36'
    }

    request = Request(url, headers=headers)
    return request

def get_content(request):
    response = urlopen(request)
    content = response.read().decode('utf-8')
    return content

def down_load(content):
    tree = etree.HTML(content)
    img_src = tree.xpath("//div[contains(@class, 'tupian-list com-img-txt-list')]/div[contains(@class, 'item')]/img/@data-original")
    img_name = tree.xpath("//div[contains(@class, 'tupian-list com-img-txt-list')]/div[contains(@class, 'item')]/img/@alt")

    for i in range(len(img_src)):
        name = img_name[i]
        img_url = img_src[i]  # 此时图片的url少了http协议
        img_url = 'https:' + img_url

        # 下载图片
        urlretrieve(url=img_url, filename='../data/imgs/' + name + '.jpg')


if __name__ == '__main__':
    start_page = int(input('请输入起始页码：'))
    end_page = int(input('请输入结束页码：'))

    for page in range(start_page, end_page + 1):
        request = create_request(page)
        content = get_content(request)
        down_load(content)

## jsonpath

### jsonpath练习

In [22]:
# 教程链接：https://blog.csdn.net/luxideyao/article/details/77802389 
import json
import jsonpath

with open('../data/jsonpath_example.json', 'r', encoding='utf-8') as f:
    obj = json.load(f)

# 书店所有书的作者
author_list = jsonpath.jsonpath(obj, '$.store.book[*].author')
author_list

# 所有作者
authors = jsonpath.jsonpath(obj, '$..author')
authors

# store下面的所有元素   
tag_list = jsonpath.jsonpath(obj, '$.store.*')
tag_list

# store下面的所有price
price_list = jsonpath.jsonpath(obj, '$.store..price')
price_list

# 最后有一本书
last_book = jsonpath.jsonpath(obj, '$..book[(@.length-1)]')
last_book

# 过滤出所有的包含isbn的书
isbn_list = jsonpath.jsonpath(obj, '$..book[?(@.isbn)]')
isbn_list

# 过滤出价格低于10的书
low_price_list = jsonpath.jsonpath(obj, '$..book[?(@.price<10)]')
low_price_list

[{'category': '修真', 'author': '六道', 'title': '坏蛋是怎样练成的', 'price': 8.95},
 {'category': '修真',
  'author': '唐家三少',
  'title': '斗罗大陆',
  'isbn': '0-553-21311-3',
  'price': 8.99}]

### 案例：淘票票

In [35]:
import re

url = "https://dianying.taobao.com/cityAction.json?activityId&_ksTS=1739379951943_108&jsoncallback=jsonp109&action=cityAction&n_s=new&event_submit_doGetAllRegion=true"


headers = {
    "content-encoding": "gzip",
    "content-language": "zh-CN",
    "content-type": "text/html;charset=UTF-8",
    "date": "Wed, 12 Feb 2025 17:05:51 GMT",
    "eagleeye-traceid": "2150434017393799518416169e00ec",
    "s": "STATUS_NOT_EXISTED",
    "server": "Tengine/Aserver",
    "strict-transport-security": "max-age=31536000",
    "timing-allow-origin": "*",
    "vary": "Accept-Encoding",
    # ":authority": "dianying.taobao.com",
    # ":method": "GET",
    # ":path": "/cityAction.json?activityId&_ksTS=1739379951943_108&jsoncallback=jsonp109&action=cityAction&n_s=new&event_submit_doGetAllRegion=true",
    # ":scheme": "https",
    "accept": "text/javascript, application/javascript, application/ecmascript, application/x-ecmascript, */*; q=0.01",
    # "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "zh,zh-CN;q=0.9",
    "bx-v": "2.5.28",
    "cookie": "t=41f6894a466c20b2deeb5ec943460b60; cookie2=13a28f469ddc88e8e87e8e7d6600a54c; v=0; _tb_token_=e85537438eab4; xlly_s=1; isg=BJCQTRj5w7-Swp-77reSMcPdYd7iWXSj6XNrwYpjk-u-xTFvMmxdMev7nYUlECx7",
    "priority": "u=1, i",
    "referer": "https://dianying.taobao.com/",
    "sec-ch-ua": "\"Not(A:Brand\";v=\"99\", \"Google Chrome\";v=\"133\", \"Chromium\";v=\"133\"",
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": "\"Windows\"",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36",
    "x-requested-with": "XMLHttpRequest"
}

request = Request(url, headers=headers)
response = urlopen(request)
content = response.read().decode('utf-8')
match = re.search('\(([^)]+)\)', content)
content = match.group(1)

with open('../data/taopiaopiao.json', 'w', encoding='utf-8') as f:
    f.write(content)

with open('../data/taopiaopiao.json', 'r', encoding='utf-8') as f:
    obj = json.load(f)

city_list = jsonpath.jsonpath(obj, "$..regionName")
city_list

['阿坝',
 '阿克苏',
 '阿拉尔',
 '阿拉善',
 '阿勒泰',
 '安康',
 '安庆',
 '鞍山',
 '安顺',
 '安阳',
 '白城',
 '百色',
 '白沙',
 '白山',
 '白银',
 '保定',
 '宝鸡',
 '保山',
 '保亭自治县',
 '包头',
 '巴彦淖尔',
 '巴中',
 '北海',
 '北京',
 '蚌埠',
 '本溪',
 '毕节',
 '滨州',
 '亳州',
 '巴音郭楞',
 '沧州',
 '长春',
 '常德',
 '昌吉',
 '昌江',
 '长沙',
 '长治',
 '常州',
 '巢湖市',
 '朝阳',
 '潮州',
 '承德',
 '成都',
 '澄迈县',
 '郴州',
 '赤峰',
 '池州',
 '重庆',
 '崇左',
 '楚雄',
 '滁州',
 '大理',
 '大连',
 '儋州',
 '丹东',
 '大庆',
 '大同',
 '大兴安岭',
 '达州',
 '屯昌',
 '德宏',
 '德阳',
 '德州',
 '定安',
 '定西',
 '迪庆',
 '东方',
 '东莞',
 '东营',
 '鄂尔多斯',
 '恩施',
 '鄂州',
 '防城港',
 '佛山',
 '抚顺',
 '阜新',
 '阜阳',
 '抚州',
 '福州',
 '甘南',
 '赣州',
 '甘孜',
 '巩义市',
 '广安',
 '广元',
 '广州',
 '贵港',
 '桂林',
 '贵阳',
 '固原',
 '哈尔滨',
 '海北',
 '海东',
 '海口',
 '海南州',
 '海西',
 '哈密',
 '韩城市',
 '邯郸',
 '杭州',
 '汉中',
 '鹤壁',
 '河池',
 '合肥',
 '鹤岗',
 '黑河',
 '衡水',
 '衡阳',
 '和田',
 '河源',
 '菏泽',
 '贺州',
 '红河',
 '淮安',
 '淮北',
 '怀化',
 '淮南',
 '黄冈',
 '黄南',
 '黄山',
 '黄石',
 '呼和浩特',
 '惠州',
 '葫芦岛',
 '呼伦贝尔',
 '湖州',
 '佳木斯',
 '吉安',
 '江门',
 '焦作',
 '嘉兴',
 '嘉峪关',
 '揭阳',
 '吉林',
 '济南',
 '金昌',
 '晋城',
 '景德镇',
 '荆门

## BeautifulSoup

In [25]:
from bs4 import BeautifulSoup


with open('../data/bs4_example.html', 'r', encoding='utf-8') as f:
    soup = BeautifulSoup(f, 'lxml')

# 根据标签名查找节点（找到的是第一个符合条件的数据）
soup.a
# 获取标签的属性和属性值
soup.a.attrs

# bs4 的一些函数
# (1) find()  返回第一个符合条件的数据
soup.find('a')
soup.find('a', title='a2')
soup.find('a', class_='a1')  # 根据class 属性查找，class需要加上下划线

# (2) find_all()  返回所有符合条件的数据，列表形式返回
soup.find_all('a')  
soup.find_all(['a', 'span'])  # 获取多个标签数据，参数为列表
soup.find_all('li', limit=2)  # 查找前2个li标签

# (3) select()  推荐
soup.select('a')
soup.select('.a1')  # 根据class属性查找(.class)
soup.select('#l1')  # 根据id属性查找(#id)


# 属性选择器（通过属性来查找对应的标签）
soup.select('li[id]')  # 查找到li标签中有id的标签
soup.select('li[id="l1"]')  # 查找到id为l1的li标签


# 层级选择起
soup.select('div li')  # 后代选择器（找到的是div下面的li）
soup.select('div > ul > li')  # 子选择器（找到的是div下的直接子标签li）

soup.select('a, li')  # 获取多个标签数据


# 节点信息（获取节点内容）
obj = soup.select('#d1')[0]
obj.string  # 只适用于标签中只有内容，没有嵌套其他标签
obj.get_text()  # 适用于标签中只有内容或包含嵌套标签（推荐）

# 节点属性
obj = soup.select('#p1')[0]
obj.name  # 获取标签名
obj.attrs # 将属性值作为字典返回

# 获取节点的属性
obj = soup.select('#p1')[0]
obj.attrs.get('class')  # 推荐
obj.get('class')
obj['class']

['p1']

### 案例：证券之星主页板块名称下载

In [ ]:
from urllib.request import urlopen, Request

url = "https://quote.stockstar.com/"

headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, likeGecko) Chrome/74.0.3729.169 Safari/537.36'
    }

req = Request(url, headers=headers)
response = urlopen(req)
content = response.read().decode('gb2312')

from bs4 import BeautifulSoup

soup = BeautifulSoup(content, 'lxml')

# xpath路径：//li/a[starts-with(@href, '/stock/industry_')]
# [attr=value] 选择 attr 属性值等于 value 的元素
# [attr*=value] 选择 attr 属性值包含 value 的元素
# [attr$=value] 选择 attr 属性值以 value 结尾的元素
# [attr~=value] 选择 attr 属性值包含单词 value 的元素（以空格分隔）
# [attr^=value] 选择 attr 属性值以 value 开头的元素（以连字符分隔）
name_list = soup.select('li a[href^="/stock/industry_"]')
name_list = [name.get_text() for name in name_list]
name_list

['・农林牧渔业',
 '・采矿业',
 '・制造业',
 '・水电煤',
 '・建筑业',
 '・批发和零售业',
 '・交通运输仓储',
 '・住宿和餐饮业',
 '・信息技术业',
 '・金融业',
 '・房地产业',
 '・租赁商务服务业',
 '・科学技术服务业',
 '・公共设施管理业',
 '・居民服务修理业',
 '・教育',
 '・卫生和社会工作',
 '・文化体育娱乐业',
 '・综合']